# Сравнение музыкальных предпочтений жителей Москвы и Санкт-Петербурга

На данных Яндекс.Музыки сравним поведение пользователей двух столиц.

**Цель исследования** — проверить гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.
 



## Обзор данных



Импортируем необходимую для работы библиотеку.

In [ ]:
import pandas as pd 

Прочтем файл `yandex_music_project.csv` и сохраним его в переменной `df`:

In [ ]:
df = pd.read_csv ('/datasets/yandex_music_project.csv') 

Выведем на экран первые десять строк таблицы:

In [ ]:
display (df.head(10)) 

Просмотрим общую информацию о таблице:

In [ ]:
display (df.info()) 

В таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Названия написано слитно, без разделителей.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

## Предобработка данных

### Стиль заголовков
Просмотрим названия столбцов:

In [ ]:
df.columns 

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

In [ ]:
df = df.rename(
    columns={
        '  userID' : 'user_id',
        'Track' : 'track',
        '  City  ' : 'city',
        'Day' : 'day',
    }
) 

Проверим результат.

In [ ]:
df.columns 

### Пропуски значений
Посчитаем, сколько в таблице пропущенных значений. 

In [ ]:
df.isna().sum() 

Не все пропущенные значения влияют на исследование. В `track` и `artist` пропуски не важны для работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [ ]:
columns_to_replace = ['track', 'artist', 'genre'] 
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown') "

Убедимся, что в таблице не осталось пропусков.

In [ ]:
df.isna().sum() 

### Дубликаты
Посчитаем явные дубликаты в таблице:

In [ ]:
df.duplicated().sum()

Удалим явные дубликаты:

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)е было пропусков в индексах

Убедимся, что избавились от дубликатов:

In [ ]:
df.duplicated().sum() 

Теперь избавимся от неявных дубликатов в колонке `genre`.

In [ ]:
df['genre'].sort_values().unique() # с помощью метода unique() выполняю проверку на неявные дубликаты

In [ ]:
def replace_wrong_genres(wrong_genres, correct_genre): # создаю функцию по замене неявных дубликатов
    for genre in wrong_genres: 
        df['genre'] = df['genre'].replace(genre, correct_genre)

In [ ]:
wrong_genres = ['hip', 'hop', 'hip-hop'] 
replace_wrong_genres(wrong_genres, 'hiphop') 

Проверим, что заменили неправильные названия:

In [ ]:
df['genre'].sort_values().unique()

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Были исправлены заголовки, чтобы упростить работу с таблицей, удалены дубликаты.

Пропущенные значения вы заменили на `'unknown'`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

In [ ]:
df.groupby('city')['track'].count()

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу.

In [ ]:
df.groupby('day')['track'].count() 

В среднем пользователи из двух городов менее активны по средам.

Теперь напишем функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

In [ ]:
def number_tracks(day, city): # создаю функцию, которая с помощью логической индексации и метода count() подсчитывает количество прослушиваний в зависимости от города и дня недели
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count ()
    return track_list_count

Вызовем `number_tracks()`, меняя значение параметров, чтобы получить данные для каждого города в каждый из трёх дней.

In [ ]:
number_tracks('Monday', 'Moscow') #с помощью функции считаю количество прослушиваний в Москве по понедельникам

In [ ]:
number_tracks('Monday', 'Saint-Petersburg') # с помощью функции считаю количество прослушиваний в Санкт-Петербурге по понедельникам

In [ ]:
number_tracks('Wednesday', 'Moscow') # с помощью функции считаю количество прослушиваний в Москве по средам

In [ ]:
number_tracks('Wednesday', 'Saint-Petersburg') # с помощью функции считаю количество прослушиваний в Санкт-Петербурге по средам

In [ ]:
number_tracks('Friday', 'Moscow') # с помощью функции считаю количество прослушиваний в Москве по пятницам 

In [ ]:
number_tracks('Friday', 'Saint-Petersburg') # с помощью функции считаю количество прослушиваний в Санкт-Петербурге по пятницам

<div class="alert alert-block alert-success">
<b>✔️ Комментарий ревьюера:</b> Всё работает и считает верно!
    </div>

Создадим таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [ ]:
columns = ['city', 'monday', 'wednesday', 'friday'] 
data = [['Moscow', 15740, 11056, 15945], 
        ['Saint-Petersburg', 5614, 7003, 5895]] 
pd.DataFrame (data=data, columns=columns) 

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [ ]:
moscow_general = df[df['city'] == 'Moscow'] 

In [ ]:
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [ ]:
def genre_weekday (table, day, time1, time2): # создаю функцию, которая считает для заданного временного промежутка топ-10 популярных жанров
    
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False) 
    
    return genre_df_sorted.head(10) 

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [ ]:
genre_weekday (moscow_general, 'Monday', '07:00', '11:00') # вызываю функцию для утра понедельника в Москве

In [ ]:
genre_weekday (spb_general, 'Monday', '07:00', '11:00') # вызываю функцию для утра понедельника в Петербурге

In [ ]:
genre_weekday (moscow_general, 'Friday', '17:00', '23:00') # вызываю функцию для вечера пятницы в Москве

In [ ]:
genre_weekday (spb_general, 'Friday', '17:00', '23:00') # вызываю функцию для вечера пятницы в Петербурге

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. 

In [ ]:
moscow_genres = moscow_general.groupby('genre')['track'].count().sort_values(ascending=False) 

In [ ]:
display (moscow_genres.head(10)) 

Теперь повторим то же и для Петербурга.

In [ ]:
spb_genres = spb_general.groupby('genre')['track'].count().sort_values(ascending=False) 

In [ ]:
display (spb_genres.head(10)) 

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Были проверены три гипотезы и установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

In [ ]:
# Исследование надёжности заёмщиков

## Данные 📁

Входные данные от банка — статистика о платёжеспособности клиентов.

## Задачи 📝

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

## Библиотеки 🛠️

`Python` `pymystem3` `Pandas` `NumPy`

<br>

<a href="https://colab.research.google.com/github/dsibi/yandex_praktikum_da/blob/main/assess_reliability_of_bank_borrowers/assess_reliability_of_bank_borrowers_ru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>